In [ ]:
!pip install datasets transformers torch accelerate hnswlib sentencepiece sentence-transformers

  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.0 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=196380 sha256=dd7dfcd1e5ef179124f0b0c4b711a7d28d3c1f1125999407f0a6a277339a8113
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [ ]:
!python -m pip install hnswlib

In [ ]:
import datasets
dataset_name = "medalpaca/medical_meadow_medical_flashcards"

dataset_flashcards = datasets.load_dataset(dataset_name)

Generating train split: 0 examples [00:00, ? examples/s]

paragraph indexing

In [ ]:
passages = [flashcard["output"] for flashcard in dataset_flashcards["train"]]

print(f"Retrieved {len(passages)} passages")

Retrieved 33955 passages


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder

semb_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
xenc_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import os
import pickle

# Define hnswlib index path
embeddings_cache_path = "./qa_embeddings_cache.pkl"

# Load cache if available
if os.path.exists(embeddings_cache_path):
    print("Loading embeddings cache")
    with open(embeddings_cache_path, "rb") as f:
        corpus_embeddings = pickle.load(f)
# Else compute embeddings
else:
    print("Computing embeddings")
    corpus_embeddings = semb_model.encode(
        passages, convert_to_tensor=True, show_progress_bar=True
    )
    # Save the index to a file for future loading
    print(f"Saving index to: '{embeddings_cache_path}'")
    with open(embeddings_cache_path, "wb") as f:
        pickle.dump(corpus_embeddings, f)

Computing embeddings


Batches:   0%|          | 0/1062 [00:00<?, ?it/s]

Saving index to: './qa_embeddings_cache.pkl'


In [ ]:
import os
import hnswlib

# Create empthy index
index = hnswlib.Index(space="cosine", dim=384)

# Define hnswlib index path
index_path = "./qa_hnswlib.index"

# Load index if available
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print("Started creating HNSWLIB index")
    index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=400, M=64)
    #  Compute the HNSWLIB index (it may take a while)
    index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
    # Save the index to a file for future loading
    print(f"Saving index to: {index_path}")
    index.save_index(index_path)

Started creating HNSWLIB index
Saving index to: ./qa_hnswlib.index


# Load the model

In [ ]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

device = "cuda"

# Load the tokenizer and model
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)
model.to(device)

# Function to generate answers
def generate_answer(question, max_length=100):
    prompt = f"Q: {question}\nA:"
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer[len(prompt) :].strip()  # Remove the prompt from the generated text

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

## Test the model

In [ ]:
answer = generate_answer("What is the appropriate treatment for a newborn with unconjugated bilirubin and high hemoglobin levels?", max_length=60)
print(answer)

The treatment of newborns with unconjugated bilirubin and high hemoglobin levels is a very important step in the treatment of congenital heart disease.


# Documents retrieval

In [ ]:
import random

random.seed(19216)

idx = random.choice(range(len(dataset_flashcards["train"])))

sample = dataset_flashcards["train"][idx]
question = sample["input"]
target_answer = sample["output"]

print(f"Question {idx}: {question}")

Question 22142: What is the appropriate treatment for a newborn with unconjugated bilirubin and high hemoglobin levels?


In [ ]:
question_embedding = semb_model.encode(question, convert_to_tensor=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
corpus_ids, distances = index.knn_query(question_embedding.cpu(), k=64)
scores = 1 - distances

print("Cosine similarity model search results")
print(f'Query: "{question}"')
print("---------------------------------------")
for idx, score in zip(corpus_ids[0][:5], scores[0][:5]):
    print(f'Score: {score:.4f}\nDocument: "{passages[idx]}"\n\n')

Cosine similarity model search results
Query: "What is the appropriate treatment for a newborn with unconjugated bilirubin and high hemoglobin levels?"
---------------------------------------
Score: 0.7628
Document: "A newborn with unconjugated bilirubin and high hemoglobin levels may require a transfusion."


Score: 0.6847
Document: "Physiologic jaundice of the newborn results in an increase in unconjugated bilirubin levels."


Score: 0.6543
Document: "When a 1-day-old baby has a total bilirubin level greater than 20 mg/dL, it is typically managed with exchange transfusion. Exchange transfusion is a medical procedure in which some of the baby's blood is removed and replaced with donor blood. This helps to reduce the level of bilirubin in the baby's blood and prevent the development of kernicterus, a rare but serious condition that can cause brain damage. In addition to exchange transfusion, other treatments may also be used to manage high bilirubin levels in newborns, such as photothe

In [ ]:
import numpy as np

model_inputs = [(question, passages[idx]) for idx in corpus_ids[0]]
cross_scores = xenc_model.predict(model_inputs)

print("Cross-encoder model re-ranking results")
print(f'Query: "{question}"')
print("---------------------------------------")
for idx in np.argsort(-cross_scores)[:5]:
    print(
        f'Score: {cross_scores[idx]:.4f}\nDocument: "{passages[corpus_ids[0][idx]]}"\n\n'
    )

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Cross-encoder model re-ranking results
Query: "What is the appropriate treatment for a newborn with unconjugated bilirubin and high hemoglobin levels?"
---------------------------------------
Score: 7.4565
Document: "A newborn with unconjugated bilirubin and high hemoglobin levels may require a transfusion."


Score: 4.2512
Document: "According to medical guidelines, phototherapy is the recommended treatment for a 4-day-old baby with a total bilirubin level of 21."


Score: 4.1218
Document: "Phototherapy treats neonatal jaundice by isomerizing unconjugated bilirubin (UCB) to a water-soluble form."


Score: 4.0761
Document: "The recommended initial treatment for rapidly rising hyperbilirubinemia in a newborn to prevent kernicterus is exchange transfusion. Hyperbilirubinemia is a common condition in newborns that occurs when there is an excess of bilirubin in the blood. In some cases, the bilirubin levels can rise rapidly and reach dangerous levels, which can lead to a condition called k

# Test the model with context

In [ ]:
passage_idxs = np.argsort(-cross_scores)[:5]
passage = "\n".join([passages[corpus_ids[0][passage_idx]] for passage_idx in passage_idxs])

input_text = f"Given the following passages, answer the related question.\n\nPassages:\n\n{passage}\n\nQ: {question}"
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
attention_mask = torch.ones(inputs.shape, device=device)
print(input_text, "\n")

outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_length=500,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
output_text = output_text[len(input_text) :].strip()
print(output_text, "\n")

print(f"A (target): {target_answer}")

Given the following passages, answer the related question.

Passages:

A newborn with unconjugated bilirubin and high hemoglobin levels may require a transfusion.
According to medical guidelines, phototherapy is the recommended treatment for a 4-day-old baby with a total bilirubin level of 21.
Phototherapy treats neonatal jaundice by isomerizing unconjugated bilirubin (UCB) to a water-soluble form.
The recommended initial treatment for rapidly rising hyperbilirubinemia in a newborn to prevent kernicterus is exchange transfusion. Hyperbilirubinemia is a common condition in newborns that occurs when there is an excess of bilirubin in the blood. In some cases, the bilirubin levels can rise rapidly and reach dangerous levels, which can lead to a condition called kernicterus, a type of brain damage. Exchange transfusion is a procedure in which a small amount of the baby's blood is removed and replaced with donor blood or a blood substitute. This helps to rapidly reduce the levels of bilirub

In [ ]:
def qa_pipeline(
    question,
    similarity_model=semb_model,
    embeddings_index=index,
    re_ranking_model=xenc_model,
    generative_model=model,
    device=device,
):
    if not question.endswith("?"):
        question = question + "?"
    # Embed question
    question_embedding = semb_model.encode(question, convert_to_tensor=True)
    # Search documents similar to question in index
    corpus_ids, distances = index.knn_query(question_embedding.cpu(), k=64)
    # Re-rank results
    xenc_model_inputs = [(question, passages[idx]) for idx in corpus_ids[0]]
    cross_scores = xenc_model.predict(model_inputs)
    # Get best matching passage
    passage_idxs = np.argsort(-cross_scores)[:5]
    passage = "\n".join([passages[corpus_ids[0][passage_idx]] for passage_idx in passage_idxs])

    input_text = f"Given the following passages, answer the related question.\n\nPassages:\n\n{passage}\n\nQ: {question}"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=500,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = output_text[len(input_text) :].strip()

    # Return result
    return f"Passages:\n\n{passage}\n\nQ: {question}\n\nA: {output_text}"

# Finetune the model

In [ ]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the dataset
dataset_name = "medalpaca/medical_meadow_medical_flashcards"
dataset = load_dataset(dataset_name)

dataset = dataset["train"].shuffle(42)
print(len(dataset))

# Split the dataset into train and validation sets (80-20 split)
train_testvalid = dataset.train_test_split(test_size=0.3)
test_valid = train_testvalid['test'].train_test_split(test_size=0.3)

# Create a DatasetDict with the train and validation sets
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

2024-05-20 11:30:15.064850: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 11:30:15.064957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 11:30:15.198110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


33955


In [ ]:
# Load the tokenizer and model
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model_finetune = GPTNeoForCausalLM.from_pretrained(model_name)

# Set padding token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Preprocess the dataset
def preprocess_function(examples):
    inputs = [
        "Q: " + question + "\nA: " + answer
        for question, answer in zip(examples["input"], examples["output"])
    ]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_datasets = train_test_valid_dataset.map(
    preprocess_function, batched=True
)
# Fine-tuning the model
training_args = TrainingArguments(
    report_to='none',
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    save_strategy="no",
    fp16=True
)

trainer = Trainer(
    model=model_finetune,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

trainer.train()

# Save the model
tokenizer.save_pretrained("./fine_tuned_model_tokenizer")
model_finetune.save_pretrained("./fine_tuned_model")


Map:   0%|          | 0/23768 [00:00<?, ? examples/s]

Map:   0%|          | 0/3057 [00:00<?, ? examples/s]

Map:   0%|          | 0/7130 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.493500,0.393739
1000,0.378200,0.370138
1500,0.368700,0.354795
2000,0.346800,0.344479
2500,0.335400,0.335924
3000,0.325300,0.327993
3500,0.318000,0.324113
4000,0.309100,0.316415
4500,0.318100,0.310441
5000,0.297800,0.306173


In [ ]:
def qa_pipeline_finetuned(
    question,
    similarity_model=semb_model,
    embeddings_index=index,
    re_ranking_model=xenc_model,
    generative_model=model,
    device=device,
):
    if not question.endswith("?"):
        question = question + "?"
    # Embed question
    question_embedding = semb_model.encode(question, convert_to_tensor=True)
    # Search documents similar to question in index
    corpus_ids, distances = index.knn_query(question_embedding.cpu(), k=64)
    # Re-rank results
    xenc_model_inputs = [(question, passages[idx]) for idx in corpus_ids[0]]
    cross_scores = xenc_model.predict(model_inputs)
    # Get best matching passage
    passage_idxs = np.argsort(-cross_scores)[:5]
    passage = "\n".join([passages[corpus_ids[0][passage_idx]] for passage_idx in passage_idxs])

    input_text = f"Q: {question}\nA:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    outputs = model_finetune.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=500,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = output_text[len(input_text) :].strip()

    # Return result
    return f"Passages:\n\n{passage}\n\nQ: {question}\n\nA: {output_text}"